In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import time
from collections import *
tf.__version__

'1.6.0'

In [30]:
lines  = open('movie_lines.txt',encoding = 'utf-8',errors = 'ignore').read().split('\n')
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [31]:
new_line = []
_line = []
for line in lines:
  _line = line.split(' +++$+++ ')
  if(len(_line)==5):
    new_line.append(_line[4])

In [32]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = re.sub(r"let's","let us",text)
  
    return text

In [33]:
clean_lines = []
for line in new_line:
    clean_lines.append(clean_text(line))
lines = clean_lines
lines[:10]

['they do not',
 'they do to',
 'i hope so',
 'she okay',
 'let us go',
 'wow',
 'okay  you are gonna need to learn how to lie',
 'no',
 'i am kidding  you know how sometimes you just become this persona  and you do not know how to quit',
 'like my fear of wearing pastels']

In [76]:
vocab = {}
count = 0
for line in lines:
    for word in line.split():
        if word not in vocab:
            vocab[word] = count
            count = count + 1
    else:
        continue
count

68421

In [77]:
df = pd.read_excel('Education.xls')
len(df)

2222

In [78]:
for i in range(len(df)):
    clean_text(df['Link'][i])

In [79]:
for i in range(len(df)):
    for word in df['Link'][i].split():
        if word not in vocab:
            vocab[word] = count+1
            count += 1
        else:
            continue

In [80]:
count

72410

In [81]:
df = pd.read_excel('Music and Dance.xls')
len(df)

2808

In [82]:
for i in range(len(df)):
    clean_text(df['Link'][i])

In [83]:
for i in range(len(df)):
    for word in df['Link'][i].split():
        if word not in vocab:
            vocab[word] = count+1
            count += 1
        else:
            continue

In [84]:
count

76980

In [85]:
vocab = sorted(vocab.keys())

# MODEL DEFINITION

In [88]:
import keras 
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Flatten , Dense , Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.models import load_model
import matplotlib.pyplot as plt

In [91]:
model = Sequential()
model.add(Dense(32,input_shape=(32,1),activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(20,activation = 'relu'))
model.add(Dense(20,activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(12,activation = 'relu'))
model.add(Dense(12,activation = 'relu'))
model.add(Dropout(0.50))

model.add(Dense(2,activation = 'softmax'))

In [92]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32, 32)            64        
_________________________________________________________________
dense_9 (Dense)              (None, 32, 32)            1056      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 32, 20)            660       
_________________________________________________________________
dense_11 (Dense)             (None, 32, 20)            420       
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 20)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 32, 12)            252       
__________